In [1]:
def ADP_Log(fatherLogger, childLogger, level, level_action, log_level, *message):
    """ function that add logs in the log file used by databricks
        Parameters:
            fatherLogger  -- It refers to the name of the one of the main processes (ADP -> general, FMT_SL -> Farmatic SellOut, FMT_ING -> Farmatic Ingestion). Has to be created in log4j.properties
            childLogger   -- Name of the logger to get or to be created. It will be child of the former logger and ususally will be the name of the notebook
            level         -- It will be the default level of the logger but if we need to change this level we have to change the log_level parameter of the notebook from DataFactory.
            level_action  -- SET or GET. Normally it would be GET but if we need to change it we have to change the level_action parameter of the notebook from DataFactory  
            log_level     -- The level to SET if SET is passed as parameter in level_action
            message       -- Message to be printed for the log. It's a tuple. 
      
        Return:
            Boolean

        Example 1:
            In order to use this function, you have to call it in this way: 
            ADP_Log("FMT_SL", "FMT_Data_TobeProcessed_Sellout", "INFO", "GET", "DEBUG", My message to print in the log file")
    """
    #Who                 When           What
    #Jose A. Latorre     27/02/2019     Initial Version
    #Jose A. Latorre     07/03/2019     Modified in order to be able of implement it in a standar way through the whole project 
    #Jose A. Latorre     11/03/2019     Modified the inheritance of Loggers. Now all loggers created in log4j.properties are at the same level
    
    try:
        
        levelObject = sc._jvm.org.apache.log4j.Level
        logManagerObject = sc._jvm.org.apache.log4j.LogManager
        
        AllLevel = levelObject.ALL
        TraceLevel = levelObject.TRACE
        DebugLevel = levelObject.DEBUG
        InfoLevel = levelObject.INFO
        WarnLevel = levelObject.WARN
        ErrorLevel = levelObject.ERROR
        FatalLevel = levelObject.FATAL
        OffLevel = levelObject.OFF
        
        # Name of one of the main process. We add "_Logger" in order to match the logger created in log4j.properties file
        fatherLogger = fatherLogger + "_Logger"
        
        loggerName = fatherLogger + "." + childLogger
        
        LEVEL = log_level.upper() # It will be used to set the level for the logger
        level = level.lower() # it will be used as function to print in the lo file
        
        message = str(message)
  
        # Get the logger (or Create if not exist)
        myLogger = logManagerObject.getLogger(loggerName)
        
        # Change the default level to be logged if needed
        if level_action.upper() == "SET":
            # convert the string passed as level in a level object. If there is any problem, DebugLevel will be setted by default
            levelToSet = levelObject.toLevel(LEVEL, DebugLevel)
        else:
            # ensure that a Father's level is setted. Father level is default level
            levelToSet = logManagerObject.getLogger(fatherLogger).getLevel()
        
        myLogger.setLevel(levelToSet)
        
        # print the message in the log file depending on the logLevel Setted
        write_logFile = "myLogger." + level + "(" + message + ")"           
        eval(write_logFile)

        return True
    
    except Exception as e:
        myLogger.error("Exception in logging (ADP_Log function). See the error: " + str(e))
        return False
      
##################################################################################################################################################################

def ADP_log_info (process, logger_name, level_action, log_level,  errorMessage, nameFunction):
  """ function that add a INFO log in the log file used by databricks
      Parameters:
          process      -- It refers to the name of the one of the main processes (ADP -> general, FMT_SL -> Farmatic SellOut, FMT_ING -> Farmatic Ingestion). Has to be created in log4j.properties
          logger_name  -- Name of the logger to get or to be created. It will be child of the former logger and ususally will be the name of the notebook
          level_action -- SET or GET. Normally it would be GET but if we need to change it we have to change the level_action parameter of the notebook from DataFactory  
          log_level    -- The level to SET if SET is passed as parameter in level_action
          errorMessage -- Message to be concatenated before to the standard message of a exception
          nameFunction -- sys._getframe().f_code.co_name (name of function that invoque ADP_log_info function)

      Return:
          Boolean

      Example 1:
          In order to use this function, you have to call it in this way: 
          ADP_log_info("FMT_SL", "FMT_Data_TobeProcessed_Sellout", "GET", "DEFAULT", "BEGIN", sys._getframe().f_code.co_name)
          ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name) 
  """
  #Who                 When           What
  #Ana Perez           21/03/2019     Initial version
  ################################################################################

  try:
    
    if nameFunction == "<module>": nameFunction = "Main"
    ADP_Log(process, logger_name, "INFO", level_action, log_level, nameFunction + ": " + errorMessage)
    
  except Exception as err:
    raise err

##################################################################################################################################################################

def ADP_log_exception (process, logger_name, level_action, log_level, errorMessage, nameFunction, sysInfo):
  """ function that add a EXCEPTION log in the log file used by databricks
      Parameters:
          process      -- It refers to the name of the one of the main processes (ADP -> general, FMT_SL -> Farmatic SellOut, FMT_ING -> Farmatic Ingestion). Has to be created in log4j.properties
          logger_name  -- Name of the logger to get or to be created. It will be child of the former logger and ususally will be the name of the notebook
          level_action -- SET or GET. Normally it would be GET but if we need to change it we have to change the level_action parameter of the notebook from DataFactory  
          log_level    -- The level to SET if SET is passed as parameter in level_action
          errorMessage -- Message to be concatenated before to the standard message of a exception
          sysInfo      -- sys.exc_info() of the exception

      Return:
          Boolean

      Example 1:
          In order to use this function, you have to call it in this way: 
          ADP_log_exception("FMT_SL", "FMT_Data_TobeProcessed_Sellout", "GET", "DEFAULT", "My message to concatenate before to the standard message of a exception", sys._getframe().f_code.co_name,  sys.exc_info())
          ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
  """
  #Who                 When           What
  #Ana Perez           21/03/2019     Initial version
  #Victor Salesa       29/04/2019     Return message to be used for other purposes
  ################################################################################

  try:
    
    if nameFunction == "<module>": nameFunction = "Main"
    exc_type, exc_value, exc_traceback = sysInfo
    error_message =  [nameFunction + ": " + errorMessage + " " ] + traceback.format_exception(exc_type, exc_value, exc_traceback, limit=None, chain=False)
    ADP_Log(process, logger_name, "ERROR", level_action, log_level, [nameFunction + ": " + errorMessage + " " ] + traceback.format_exception(exc_type, exc_value, exc_traceback, limit=None, chain=False))
    return error_message
  except Exception as err:
    raise err 

##################################################################################################################################################################    
def ADP_log_debug (process, logger_name, level_action, log_level, errorMessage, nameFunction):
  """ function that add a DEBUG log in the log file used by databricks
      Parameters:
          process      -- It refers to the name of the one of the main processes (ADP -> general, FMT_SL -> Farmatic SellOut, FMT_ING -> Farmatic Ingestion). Has to be created in log4j.properties
          logger_name  -- Name of the logger to get or to be created. It will be child of the former logger and ususally will be the name of the notebook
          level_action -- SET or GET. Normally it would be GET but if we need to change it we have to change the level_action parameter of the notebook from DataFactory  
          log_level    -- The level to SET if SET is passed as parameter in level_action
          errorMessage -- Message to be concatenated before to the standard message of a exception
          nameFunction -- sys._getframe().f_code.co_name (name of function that invoque ADP_log_info function)

      Return:
          Boolean

      Example 1:
          In order to use this function, you have to call it in this way: 
          ADP_log_debug("FMT_SL", "FMT_Data_TobeProcessed_Sellout", "GET", "Before execute a write into T_STG_SELL_OUT table", sys._getframe().f_code.co_name)
          ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical PRODUCT: " + str(pharmatic_product_master_df.count()) + " rows ", sys._getframe().f_code.co_name)
  """
  #Who                 When           What
  #Ana Perez           21/03/2019     Initial version
  ################################################################################

  try:
    
    if nameFunction == "<module>": nameFunction = "Main"
    ADP_Log(process, logger_name, "DEBUG", level_action, log_level, nameFunction + ": " + errorMessage)

  except Exception as err:
    raise err

##################################################################################################################################################################        
def ADP_log_warning (process, logger_name, level_action, log_level, errorMessage, nameFunction):
  """ function that add a WARN log in the log file used by databricks
      Parameters:
          process      -- It refers to the name of the one of the main processes (ADP -> general, FMT_SL -> Farmatic SellOut, FMT_ING -> Farmatic Ingestion). Has to be created in log4j.properties
          logger_name  -- Name of the logger to get or to be created. It will be child of the former logger and ususally will be the name of the notebook
          level_action -- SET or GET. Normally it would be GET but if we need to change it we have to change the level_action parameter of the notebook from DataFactory  
          log_level    -- The level to SET if SET is passed as parameter in level_action
          errorMessage -- Message to be concatenated before to the standard message of a exception
          nameFunction -- sys._getframe().f_code.co_name (name of function that invoque ADP_log_info function)

      Return:
          Boolean

      Example 1:
          In order to use this function, you have to call it in this way: 
          ADP_log_warning(process, logger_name, level_action, log_level, "It does not find files", sys._getframe().f_code.co_name) 
  """
  #Who                 When           What
  #Ana Perez           22/03/2019     Initial version
  ################################################################################

  try:
    
    if nameFunction == "<module>": nameFunction = "Main"
    ADP_Log(process, logger_name, "WARN", level_action, log_level, nameFunction + ": " + errorMessage)

  except Exception as err:
    raise err     